# Generate data samples

## Phase space samples

In [ ]:
from pycompwa.expertsystem.ui.system_control import StateTransitionManager
from pycompwa.expertsystem.ui.system_control import InteractionTypes
from pycompwa.expertsystem.amplitude.helicitydecay import \
    HelicityAmplitudeGeneratorXML

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
tbd_manager.allowed_intermediate_particles = ['f']

graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, _ = tbd_manager.find_solutions(graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

Now we can go through the procedure explained above:

In [ ]:
import pycompwa.ui as pwa

In [ ]:
# specify kinematics
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info())
random_generator = pwa.StdUniformRealGenerator(12345)

# generate phase space sample
phsp_sample = pwa.generate_phsp(1000, generator, random_generator)

## Intensity-based samples

In [ ]:
# create intensity profile
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity()

# generate data
data_sample = pwa.generate(
    100, kinematics, generator, intensity, random_generator)

## Importance sampling

In the example above, we used the decay $J/\psi \to \gamma \pi^0 \pi^0$ with
f resonances only. Imagine, however, that we want to investigate the
resonance $\omega \to \gamma\pi^0$. This resonance has such a narrow peak
that you will run into problems when normalizing the intensity model.

To normalize a distribution, you need know the integral over that
distribution. The integral over an intensity distribution distribution can
only be computed numerically. We simply do this by computing the sum of the
intensities evaluated for each point in a phase space (Monte Carlo
integration), and averaging over the number of phase space points. This means
that the higher the number of phase space points, the more accurate your
integral.

With a narrow peak, however, many of the points in an evenly distributed
phase space sample will lie outside the peak region, where the intensity (and
therefore their contribution to the integral) is highest. So in order to have
a good estimate for the integral over this narrow peak structure, you want to
have a larger number of phase space points under that peak. Now, you could
just use a bigger phase space sample (increasing number of points within the
peak region), but this will be resource intensive as it will result in many
hits in regions that hardly contributes to the integral.

A smarter strategy is generate a sample that has a higher density of phase
space points within the peak region. This strategy is called **importance
sampling**.

In [ ]:
from pycompwa.expertsystem.ui.system_control import StateTransitionManager
from pycompwa.expertsystem.ui.system_control import InteractionTypes
from pycompwa.expertsystem.amplitude.helicitydecay import \
    HelicityAmplitudeGeneratorXML
from pycompwa.expertsystem.state.particle import particle_list

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

# Widen omega resonance
omega = particle_list["omega(782)"]
parameters = omega['DecayInfo']['Parameter']
for par in parameters:
    if par['@Type'] == 'Width':
        par['Value'] = 0.001

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
tbd_manager.allowed_intermediate_particles = ['f2(1270)', 'omega']
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, _ = tbd_manager.find_solutions(graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

In [ ]:
import pycompwa.ui as pwa

In [ ]:
# specify kinematics
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info())
random_generator = pwa.StdUniformRealGenerator(12345)

# generate evenly distributed phase space sample
phsp_sample = pwa.generate_phsp(100000, generator, random_generator)

# create intensity
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity()

# generate importance-sampled phase space sample
phsp_sample_importance = pwa.generate_importance_sampled_phsp(
    2000, kinematics, generator, intensity, random_generator)

In [ ]:
data_sample = pwa.generate(
    2000, kinematics, generator, intensity, random_generator)

In [ ]:
# generate data
data_sample = pwa.generate(
    100, kinematics, generator, intensity, random_generator)

The effect of this importance sampling can be best seen in a Dalitz plot.
There, one can see that there are more events in the regions where the
intensity is large (the resonances), but one still sees a uniform
distribution overall.

In [ ]:
kinematics.create_all_subsystems()
phsp_set = kinematics.convert(phsp_sample)
phsp_set_importance = kinematics.convert(phsp_sample_importance)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
transition_info = kinematics.get_particle_state_transition_kinematics_info()
id_to_name = transition_info.get_final_state_id_to_name_mapping()


def replace_ids(title):
    """Replace all IDs in a string by particle names."""
    for id, name in id_to_name.items():
        title = title.replace(str(id), name)
    return title


def dalitz_plot(data_set, mass_x, mass_y, bins=50, **kwargs):
    """Helper function to create a Dalitz plot with useful axis titles."""
    plt.hist2d(
        data_set.data[mass_x],
        data_set.data[mass_y],
        weights=data_set.weights,
        bins=bins,
        **kwargs)
    plt.xlabel(replace_ids(mass_x))
    plt.ylabel(replace_ids(mass_y))

In [ ]:
dalitz_plot(phsp_set_importance, 'mSq_(3,4)', 'mSq_(2,4)',
            bins=80, norm=LogNorm())
plt.gca().set_title('importance-sampled phase space');

Note how this compares to the evenly distributed phase space sample and the
intensity-based sample:

In [ ]:
dalitz_plot(phsp_set, 'mSq_(3,4)', 'mSq_(2,4)', bins=80, norm=LogNorm())
plt.gca().set_title('evenly distributed phase space');

In [ ]:
pwa.Logging('error')
data_sample = pwa.generate(
    2000, kinematics, generator, intensity, random_generator)
data_set = kinematics.convert(data_sample)
dalitz_plot(data_set, 'mSq_(3,4)', 'mSq_(2,4)', bins=80, norm=LogNorm())
plt.gca().set_title('intensity-based sample');